<h1 style="color:#00A6D6;">Introduction to Quantum Cryptography - Jupyter Notebooks</h1>
<h2 style="color:#00A6D6;">Chapter 4: The power of entanglement </h2>

* <a href="#sep"> Separable vs Entangled states </a>

* <a href="#uhl"> Uhlman's theorem </a>

* <a href="#sch"> Schmidt Decomposition </a>

* <a href="#bel"> Bell Test </a>

Notice that the line below will make Julia add and compile several packages. After compiling them it will print several info and warning messages.

In [1]:
include("source/main.jl");

   Resolving package versions...
  No Changes to `C:\Users\swehner\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\swehner\.julia\environments\v1.9\Manifest.toml`
Precompiling project...
  ✓ AMD
  ✓ AbstractTrees
  ✓ LDLFactorizations
  ✓ Convex
  4 dependencies successfully precompiled in 8 seconds. 80 already precompiled.
   Resolving package versions...
  No Changes to `C:\Users\swehner\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\swehner\.julia\environments\v1.9\Manifest.toml`


WebIO._IJuliaInit()

<a id="sep"></a>
<h2 style="color:#00A6D6;"> Separable vs Entangled states </h2>

This week we learnt formally about entangled states. Given two quantum systems $A$ and $B$ the joint system is separable if it can be written like 

$$
\rho_{AB}=\sum_xp_x\rho_x^A\otimes\rho_x^B
$$ 

where $\{p_x\}_x$ is some probability distribution and $\{\rho_x^A\}_x$, $\{\rho_x^B\}_x$ are valid quantum states. Otherwise we say that the state is entangled. 

We have prepared for you the function check_entangled. It can decide whether a bipartite qubit state (Alice and Bob hold qubits) is entangled or not. This trick however works only if A and B are qubits - in fact, it is NP hard to find out whether an arbitrary density matrix is entangled! Let us first try it with the maximally mixed state.

In [2]:
rho = [0.25 0 0 0; 0 0.25 0 0; 0 0 0.25 0; 0 0 0 0.25]
check_entangled(rho)

false

Can you guess if the following states are entangled or not? Try first to think what should the answer be and then use the function given.

<h3 style="color:#00A6D6;"> Exercise 1</h3>

Check whether or not the following states are entangled.

1. $\rho_{AB}=\begin{pmatrix} 0.5 & 0.0 & 0.0 & -0.3 \\ 0.0 & 0.0 & 0.0 & 0.0 \\ 0.0 & 0.0 & 0.0 & 0.0 \\ -0.3 & 0.0 & 0.0 & 0.5\end{pmatrix}$
2. $\rho_{AB}=\begin{pmatrix} 0.5 & 0.0 & 0.0 & 0.0 \\ 0.0 & 0.0 & 0.0 & 0.0 \\ 0.0 & 0.0 & 0.0 & 0.0 \\ 0.0 & 0.0 & 0.0 & 0.5\end{pmatrix}$
3. $\rho_{AB}=\begin{pmatrix} 0.25 & 0.25 & 0.0 & 0.0 \\ 0.25 & 0.25 & 0.0 & 0.0 \\ 0.0 & 0.0 & 0.25 & 0.25 \\ 0.0 & 0.0 & 0.25 & 0.25\end{pmatrix}$

In [3]:
# Adapt the code from above to check for the states above
rho = [0.25 0 0 0; 0 0.25 0 0; 0 0 0.25 0; 0 0 0 0.25]
check_entangled(rho)

false

<h3 style="color:#00A6D6;"> Supplement to Exercise 1</h3>

One would imagine that checking whether a state is entangled is very simple. It turns out that this is only the case for very small dimensional systems. More concretely, in the case that both Alice and Bob hold qubits there is a very simple test. This is the test that we have implemented in the function check_entangled. Imagine that we are given 

$$
\rho_{AB}=\sum_{i,j}\lambda_{ijkl}|i\rangle\langle j|^A\otimes |k\rangle\langle l|^B,
$$ 

the test consist in checking whether the matrix $\rho_{AB}^{\Gamma_B}$ is positive or not. Specifically, with positive we mean that $\rho_{AB}^{\Gamma_B}$ is positive-semidefinite, i.e., having no negative eigenvalues. $\rho_{AB}^{\Gamma_B}$ is called the partial transpose of the $B$ system and can be computed as follows:

\begin{align}
\rho_{AB}^{\Gamma_B} &=\sum_{i,j}\lambda_{ijkl}|i\rangle\langle j|^A \otimes  T\left(|k\rangle\langle l|\right)^B\\
              &=\sum_{i,j}\lambda_{ijkl} |i\rangle\langle j|^A \otimes |l\rangle\langle k|^B
\end{align}

where $T(|i\rangle\langle j|)=|j\rangle\langle i|$. That is, $T$ corresponds with the transpose operation. If you are curious check the source code!

Correspondingly, the test is also called the partial transpose test. For qubits the test is exact: $\rho_{AB}$ is NOT entangled if and only if $\rho_{AB}^{\Gamma_B} \geq 0$. However, in general the test only gives a one-sided indicator: if NOT $\rho_{AB}^{\Gamma_B} \geq 0$, then $\rho_{AB}^{\Gamma_B}$ is entangled. However, if $\rho_{AB}^{\Gamma_B} \geq 0$ then the state can be both entangled or separable. 

<a id="uhl"></a>
<h2 style="color:#00A6D6;"> Uhlman's theorem </h2>

Later on, we will use Uhlman's theorem to cheat on a cryptographic protocol! To this end, let us delve into detail on how we can make it work towards our advantage. 
When two **rank-one** density matrices $\Phi^+_{AB}$ and $\Psi^+_{AB}$ have the same partial trace - $Tr_{A}(\Phi^+_{AB}) = Tr_A(\Psi^+_{AB})$ - it follows - in accordance with Uhlman's theorem - that the matrices are equivalent up to a local unitary on $A$. This means that if Bob has qubit $B$, then Alice can - by applying this precise unitary! - transform one state to the other. 

Let us demonstrate this with an example:

<h4 style="color:#00A6D6;"> Example </h4>

Are 

$$ \Phi^+_{AB} = \frac{1}{2} \cdot \begin{pmatrix} 1 & 0 & 0 & 1 \\ 0 & 0 & 0 & 0 \\ 0 & 0 & 0 & 0 \\ 1 & 0 & 0 & 1  \end{pmatrix} $$

and 

$$ \Psi^+_{AB} = \frac{1}{2} \cdot \begin{pmatrix} 0 & 0 & 0 & 0 \\ 0 & 1 & 1 & 0 \\ 0 & 1 & 1 & 0 \\ 0 & 0 & 0 & 0  \end{pmatrix} $$

equivalent up to a local unitary? Well, these matrices both have equal partial traces. 

$$ \frac{1}{2} \cdot \begin{pmatrix} 1 & 0 \\ 0 & 1 \end{pmatrix} \:. $$

Hence, it follows that these states are equivalent up to a local unitary. More concretely, it turns out that

$$ \Phi^+_{AB} = \big( I_A \otimes X_B \big) \cdot \Psi^+_{AB} \cdot \big( I_A \otimes X_B \big) $$

or like wise

$$ \Phi^+_{AB} = \big( X_A \otimes I_B \big) \cdot \Psi^+_{AB} \cdot \big( X_A \otimes I_B \big)\ .$$

where $X_A$ and $X_B$ are the Pauli $X$ - i.e. the bit flip operation - applied to qubits $A$ and $B$ respectively. Alice can thus apply $X_A$ to her qubit to transform one state to the other, and so can Bob by applying $X_B$. 

We have written the function partialTrace to compute the partial trace of a bipartite qubit system. In order to check that the example states have equal reduced states, this is the code that we could run:

In [4]:
PhiPlus =  [.5 0 0 .5; 0 0 0 0; 0 0 0 0; .5 0 0 .5]
partialTrace(PhiPlus,"B") # returns the reduced state on A (traces out the B system)

2×2 Matrix{Float64}:
 0.5  0.0
 0.0  0.5

In [5]:
PsiPlus =  [0 0 0 0; 0 0.5 0.5 0; 0 0.5 0.5 0; 0 0 0 0]
partialTrace(PsiPlus,"B") # returns the reduced state on A (traces out the B system)

2×2 Matrix{Float64}:
 0.5  0.0
 0.0  0.5

<h3 style="color:#00A6D6;"> Exercise 2.1</h3>

Are the matrices

$$ \rho^1_{AB} = \begin{pmatrix} \frac{1}{3} & 0.0 & 0.0 & -\frac{\sqrt{2}}{3} \\ 0.0 & 0.0 & 0.0 & 0.0 \\ 0.0 & 0.0 & 0.0 & 0.0 \\ -\frac{\sqrt{2}}{3} & 0.0 & 0.0 & \frac{2}{3}\end{pmatrix}$$

$$ \rho^2_{AB} = \begin{pmatrix} 0.0 & 0.0 & 0.0 & 0.0 \\ 0.0 & \frac{1}{3} & -\frac{\sqrt{2}}{3} & 0.0 \\ 0.0 & -\frac{\sqrt{2}}{3} & \frac{2}{3} & 0.0 \\ 0.0 & 0.0 & 0.0 & 0.0\end{pmatrix}$$

equivalent up to a local unitary? 

<h3 style="color:#00A6D6;"> Exercise 2.2</h3>

Next, are the matrices

$$ \rho^1_{AB} = \begin{pmatrix} \frac{1}{3} & 0.0 & 0.0 & -\frac{\sqrt{2}}{3} \\ 0.0 & 0.0 & 0.0 & 0.0 \\ 0.0 & 0.0 & 0.0 & 0.0 \\ -\frac{\sqrt{2}}{3} & 0.0 & 0.0 & \frac{2}{3}\end{pmatrix}$$

$$ \rho^2_{AB} = \begin{pmatrix} 0.0 & 0.0 & 0.0 & 0.0 \\ 0.0 & \frac{1}{3} & 0.0 & 0.0 \\ 0.0 & 0.0 & \frac{2}{3} & 0.0 \\ 0.0 & 0.0 & 0.0 & 0.0\end{pmatrix}$$

equivalent up to a local unitary?

In [6]:
# Your code goes here, where you may wish to work from the partialTrace example given above.

<a id="sch"></a>
<h2 style="color:#00A6D6;"> Schmidt decomposition </h2>

We have already called the EPR pair maximally entangled. But can states also be somewhat entangled? And how could we measure such partial entanglement? 

Many measures exist for entanglement - one very convenient one arises from the Schmidt decomposition. Let's see how this works!

A pure bipartite state can in general be written down as 

$$ \lvert \Phi_{AB} \rangle = \sum_{i=1}^{d_A}\sum_{j=1}^{d_B} \alpha_{ij} \lvert u_i \rangle^A \lvert v_j \rangle^B$$

with $\lvert u_i \rangle$ and $\lvert v_j \rangle$ some orthonormal basis and $\alpha_{ij}$ complex coefficients. Alternatively, any pure bipartite state can also be written as

$$ \lvert \Phi_{AB} \rangle = \sum_{i=1}^{\min\{d_A,d_B\}} \sqrt{\lambda_i} \lvert u_i \rangle \lvert v_i \rangle $$

with $\lvert u_i \rangle$, $\lvert v_i \rangle$ orthonormal bases and $\lambda_i$ non-negative coefficients that add up to one. This second form involves only $\min\{d_A,d_B\}$ coefficients $\lambda_i$ versus the $d_Ad_B$ coefficients $\alpha_{ij}$ of the first form. This is one of the attractive properties of this second form which was introduced in the videos as the Schmidt decomposition.

It's turns out that the number of non-zero Schmidt coefficients tells us something about entanglement. Let's see how we can compute these coefficients and see what they look like for some of our favorite states!

In the following we will find the coefficients of the Schmidt decomposition. Given a bipartite state $|\psi\rangle$, they can be found as follows:

1. Construct the associated density matrix $\rho = \lvert \psi \rangle \langle \psi \rvert$.
2. Compute the reduced state $\rho_A$.
3. Find the eigenvalues $\lambda_i$ of $\rho_A$.
4. The Schmidt coefficients are the square roots of the eigenvalues.

**(Bonus question)** Can you prove that this recipe really returns the Schmidt coefficients?

We can turn the recipe into Julia code as follows:

In [7]:
# input a state
psi = [1 0 0 1]/sqrt(2)

# 1. Construct the density matrix
rho = psi'*psi 

# 2. Get the reduced state
rho_A = partialTrace(rho,"B") 

# 3. Find the eigenvalues of the reduced state
lambdas = eigvals(rho_A) 

# 4. The Schmidt coefficients are the square roots of the eigenvalues
sCoef = sqrt.(lambdas) 

2-element Vector{Float64}:
 0.7071067811865475
 0.7071067811865475

<h3 style="color:#00A6D6;"> Exercise 3</h3>

Calculate the Schmidt coefficients of the following bipartite qubit states and enter them into edX. 

How many non-zero coefficients are there for each state? Are the following states entangled? Check (for example using the function above!) whether the states are entangled and how this relates to the number of non-zero Schmidt coefficients!

* $|\psi\rangle_{AB}^1=|00\rangle$
* $|\psi\rangle_{AB}^2=\frac{1}{\sqrt 2}\left(|00\rangle+|01\rangle\right)$
* $|\psi\rangle_{AB}^3=\frac{1}{\sqrt 2}\left(|00\rangle+|11\rangle\right)$
* $|\psi\rangle_{AB}^4=\sin\left(\frac{\pi}{7}\right)|00\rangle-\cos\left(\frac{\pi}{7}\right)|11\rangle$

In [8]:
# Your code goes here, where you may wish to start by copy pasting the stepwise procedure given just before

<a id="bel"></a>
<h2 style="color:#00A6D6;"> Bell Test </h2>

We saw in the chapter that entanglement allows correlations that cannot be replicated classically. We exemplified this idea with a bipartite game called CHSH. This game cannot be won classically with probability higher than $p_{\textrm{win}}=3/4$. However, if Alice and Bob (the two parties) share a maximally entangled state and choose appropriate measurement basis, then they can win with approximately $p_{\textrm{win}}\approx 0.854$.

Indeed, the CHSH inequality is the one tested in the Delft Bell experiment! Let us now imagine that Alice and Bob cannot make perfect measurements, or they have a noisy entangled state. As you might imagine, this is the more typical scenario in real world quantum implementations. How well can Alice and Bob win CHSH under such conditions?

Now, let's imagine that Alice and Bob share an arbitrary bipartite state, Alice and Bob choose measurement basis given by angles $\theta_0,\theta_1$ and $\gamma_0,\gamma_1$ respectively. What is the winning probablity? We have programmed a function that computes this winning probability (it would also be a good exercise to try to program it yourself!). 

When we say that the measurement basis is given by angle $\theta$ we mean that we measure in the basis given by 

$$\{\cos(\theta)|0\rangle+\sin(\theta)|1\rangle,-\sin(\theta)|0\rangle+\cos(\theta)|1\rangle\}.$$

In [9]:
# Let's compute the winning probability with the optimal bases and the maximally entangled state

θ_0 = 0;
θ_1 = 45;
γ_0 = 22.5;
γ_1 = -22.5;
rho_AB = [1 0 0 1; 0 0 0 0; 0 0 0 0; 1 0 0 1]/2;

pwinCHSH(rho_AB,θ_0, θ_1, γ_0, γ_1)

0.8535533905932738

<h3 style="color:#00A6D6;"> Exercise 4</h3>

Compute the winning probability for the following cases. The first two are highly relevant in an implementation: Alice and Bob would really like to share $\Phi^+$, but due to imperfections they instead end up with a mixed state $\rho_{AB}$ which with some probability is $|01\rangle\langle 01|$.

1. Shared state $\rho_{AB} = \frac{3}{4} \Phi^+ +\frac{1}{4}|01\rangle\langle 01|$, and measurement bases: $\theta_0 = 0, \theta_1 = 45, \gamma_0 = 22.5, \gamma_1 = -22.5$
2. Shared state $\rho_{AB} = \frac{1}{4} \Phi^+ +\frac{3}{4}|01\rangle\langle 01|$, and measurement bases: $\theta_0 = 0, \theta_1 = 45, \gamma_0 = 22.5, \gamma_1 = -22.5$
3. Shared state $\rho_{AB} = \Phi^+$, and measurement bases: $\theta_0 = 0, \theta_1 = 45, \gamma_0 = 0, \gamma_1 = 45$

where $\Phi^+=|\phi^+\rangle\langle\phi^+|$ and $|\phi^+\rangle=\left(|00\rangle+|11\rangle\right)/\sqrt{2}$. 

In [12]:
# Your code goes here, where you may want to start by adapting the code given just before.

<h3 style="color:#00A6D6;"> Visualization of winning probability in the CHSH game </h3>

The following snippet of code visualizes the winnning probability and probability distribution for different measurement settings given that Alice and Bob share a state of the form

$$
(1-p) \Phi^+ + p \frac{\mathbb I}{4}
$$

States of the form above are called Werner states. You can think of the term $\mathbb{I}/4$ as noise created by an imperfect appararatus!

In [13]:
@manipulate for θ_0 = [0,22.5,45], 
                θ_1 = [45,0,22.5,77.5], 
                γ_0 = [22.5,0,45,77.5], 
                γ_1 = [-22.5,0,22.5,45], 
                p = [0,0.5,1]
    
    rho_AB = p*0.25*I(4) + (1-p)*0.5*[1 0 0 1; 0 0 0 0; 0 0 0 0; 1 0 0 1] 

    # Calculate the associated probability distributions
    p00, p01, p10, p11 = bell_test_calculations(rho_AB, θ_0, θ_1, γ_0, γ_1)
    
    # Plot 
    fig = figure(figsize=(15,2))
    withfig(fig) do
        bell_test_plot(θ_0, θ_1, γ_0, γ_1, p00, p01, p10, p11)
    end
end

sys:1: UserWarning: FixedFormatter should only be used together with FixedLocator


Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["θ_0"], Dict{Symbol, Any}(:className => "interact ", :style => Dict{Any, Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol, Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol, Any}(:max => 3, :min => 1, :attributes => Dict{Any, Any}(:type => "range", Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}", "orient" => "horizontal"), :step => 1, :className => "slider slider is-fullwidth", :style => Dict{Any, Any}()))], Dict{Symbol, Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol, Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol, Any}(:className => "interact-flex-row-right"))], Dict{Symbol, Any}(:className => "interact-flex-row interact-widget")), Dict{String, Tuple{AbstractObservable, Union{Nothing, Bool}}}("changes" => (Observable(0), nothing), "index" => (Observable{Any}(2), nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "C:\\Users\\swehner\\.julia\\packages\\Knockout\\HReiN\\src\\..\\assets\\knockout.js"), Asset("js", "knockout_punches", "C:\\Users\\swehner\\.julia\\packages\\Knockout\\HReiN\\src\\..\\assets\\knockout_punches.js"), Asset("js", nothing, "C:\\Users\\swehner\\.julia\\packages\\InteractBase\\LJXv3\\src\\..\\assets\\all.js"), Asset("css", nothing, "C:\\Users\\swehner\\.julia\\packages\\InteractBase\\LJXv3\\src\\..\\assets\\style.css"), Asset("css", nothing, "C:\\Users\\swehner\\.julia\\packages\\Interact\\PENUy\\src\\..\\assets\\bulma_confined.min.css")], Dict{Any, Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")], "index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(32), Set{AbstractConnection}(), Base.GenericCondition{ReentrantLock}(Base.IntrusiveLinkedList{Task}(Task (runnable) @0x000001b8bcf75560, Task (runnable) @0x000001b8bcf75560), ReentrantLock(nothing, 0x00000000, 0x00, Base.GenericCondition{Base.Threads.SpinLock}(Base.IntrusiveLinkedList{Task}(nothing, nothing), Base.Threads.SpinLock(0)), (0, 140711831170848, 1892284695664)))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"0.0\",\"22.5\",\"45.0\"],\"changes\":WebIO.getval({\"name\":\"chang

<b> Caption: </b> From left to right: 
Alice's measurement bases represented by the red and orange lines. Red (orange) represents the measurement basis associated with setting $x=0$ ($x=1$). 

Bob's measurement bases represented by the blue (light blue) lines. Blue (light blue) represents the measurement basis associated with setting $y=0$ ($y=1$).

The probability distribution on measurement outcomes $a$, $b$ given settings $x=0$, $y=0$ - each pillar corresponds to $p(a,b|x,y)$ for $a,b = 00, 01,10,11$. Likewise for the remaining combinations of $x \in \{0,1\}$, $y \in \{0,1\}$ (from left to right). Note, the weighted sum of the probabilities associated with shaded bars give the CHSH winning probability. 